In [156]:
import pandas as pd
import numpy as np 
import datetime
inventory = pd.read_csv("data/inventory.csv")
products = pd.read_csv("data/products.csv")
promotions = pd.read_csv("data/promotions.csv")
transactions = pd.read_csv("data/transactions.csv")

import ast
for colname in inventory.columns[2:]:
    inventory[colname] = [ast.literal_eval(i) for i in inventory[colname]]

In [181]:
product = "Paprika"
transactions[transactions["description"] == product]

,day,time,customer,bank acount,category,product_id,description,size,std_sales_price,purchase_price,bio,basic
193,1/1/2018,12:20:00,14.0,67267391.0,vegetable,v_5,Paprika,1st,0.99,0.99,0.0,0.0
233,1/1/2018,13:03:28,16.0,NaN,vegetable,v_5,Paprika,1st,0.99,0.99,0.0,0.0
305,1/1/2018,13:21:22,19.0,25671204.0,vegetable,v_5,Paprika,1st,0.99,0.99,0.0,0.0
333,1/1/2018,13:37:36,21.0,88026174.0,vegetable,v_5,Paprika,1st,0.99,0.99,0.0,0.0
412,1/1/2018,14:09:57,24.0,55280334.0,vegetable,v_5,Paprika,1st,0.99,0.99,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
354373,30-12-2018,15:47:54,20663.0,87022141.0,vegetable,v_5,Paprika,1st,0.99,0.89,0.0,0.0
354605,30-12-2018,17:53:03,20673.0,21215141.0,vegetable,v_5,Paprika,1st,0.99,0.89,0.0,0.0
354701,30-12-2018,18:55:46,20678.0,83610097.0,vegetable,v_5,Paprika,1st,0.99,0.89,0.0,0.0
354751,30-12-2018,19:22:26,20680.0,9697098.0,vegetable,v_5,Paprika,1st,0.99,0.89,0.0,0.0


In [182]:
def date_converter(row):
    converted_date = datetime.datetime(2018, 1, 1) + datetime.timedelta(row["day"])
    return converted_date

inventory["date"] = inventory.apply(date_converter, axis = 1)

In [183]:
def week_of_year(row):
    week_nr = datetime.date(row["date"].year, row["date"].month, row["date"].day).isocalendar()[1]
    return week_nr

inventory["week"] = inventory.apply(week_of_year, axis = 1)

In [184]:
filter_inventory = inventory[["day", "before or after delivery", "date", "week", product]]
promotions_filter = promotions[promotions["description"] == product]

def check_discount(row):
    if row["week"] in promotions_filter["week"].values:
        discount_amount = promotions_filter[promotions_filter["week"] == row["week"]]["discount"]
        return discount_amount.values[0]
    else:
        return 0

filter_inventory["discount"] = filter_inventory.apply(check_discount, axis = 1)


In [185]:
def total_inv(row):
    total = 0
    if len(row[product]) > 0:
        for batch in row[product]:
            total += batch[1]
    return total

filter_inventory["total inventory"] = filter_inventory.apply(total_inv, axis = 1)

In [186]:
best_before = []
for element_list in filter_inventory[product]:
    for batch in element_list:
        best_before.append(batch[0])
bb_dates = np.unique(best_before)

In [187]:
df_waste = pd.DataFrame({"best before":bb_dates})
df_waste

,best before
0,14
1,17
2,21
3,24
4,28
...,...
99,360
100,364
101,367
102,371


In [188]:
def input_inv(row):
    for element_list in filter_inventory[product]:
        if len(element_list) > 0:
            for batch in element_list:
                if batch[0] == row["best before"]:
                    return batch[1]
df_waste["amount"] = df_waste.apply(input_inv, axis = 1)

In [189]:
df_waste
#product gets thrown away on the day it goes bad, no one can buy it anymore ON THAT DAY
#discard any items that has best before for enxt year
#haven't sold anything that is not in stock
df_waste = df_waste[df_waste["best before"] < 365]
df_waste

,best before,amount
0,14,34
1,17,34
2,21,34
3,24,34
4,28,14
...,...,...
96,350,34
97,353,34
98,357,34
99,360,34


In [190]:
df_waste["purchases"] = [purchases_per_day[i-1] for i in df_waste["best before"]]

In [191]:
df_waste["prev purchases"] = df_waste["purchases"].diff()
df_waste

,best before,amount,purchases,prev purchases
0,14,34,115,NaN
1,17,34,133,18.0
2,21,34,170,37.0
3,24,34,190,20.0
4,28,14,209,19.0
...,...,...,...,...
96,350,34,2840,42.0
97,353,34,2871,31.0
98,357,34,2896,25.0
99,360,34,2913,17.0


In [192]:
length = df_waste.shape[0]
lst_waste = []
df_waste["remaining"] = df_waste["amount"] - df_waste["prev purchases"]
df_waste["remaining"].iloc[0] = df_waste["amount"].iloc[0] - df_waste["purchases"].iloc[0]

for i in range(length):
    current_row = df_waste.iloc[i]
    if i == 0:
        lst_waste.append(current_row["remaining"])
        continue     
    else:
        prev_row = df_waste.iloc[i-1]
        if lst_waste[-1] < 0:
            waste = lst_waste[-1] + current_row["remaining"]
            lst_waste.append(lst_waste[-1] + current_row["remaining"])
        else:
            lst_waste.append(current_row["remaining"])
    
df_waste["waste"] = lst_waste
def date_converter(row):
    converted_date = datetime.datetime(2018, 1, 1) + datetime.timedelta(row["best before"])
    return converted_date

df_waste["date"] = df_waste.apply(date_converter, axis = 1)
df_waste["week"] = df_waste.apply(week_of_year, axis = 1)
df_waste[df_waste["waste"] < 0]["waste"] = 0

In [193]:
df_waste

,best before,amount,purchases,prev purchases,remaining,waste,date,week
0,14,34,115,NaN,-81.0,-81.0,2018-01-15,3
1,17,34,133,18.0,16.0,-65.0,2018-01-18,3
2,21,34,170,37.0,-3.0,-68.0,2018-01-22,4
3,24,34,190,20.0,14.0,-54.0,2018-01-25,4
4,28,14,209,19.0,-5.0,-59.0,2018-01-29,5
...,...,...,...,...,...,...,...,...
96,350,34,2840,42.0,-8.0,-8.0,2018-12-17,51
97,353,34,2871,31.0,3.0,-5.0,2018-12-20,51
98,357,34,2896,25.0,9.0,4.0,2018-12-24,52
99,360,34,2913,17.0,17.0,17.0,2018-12-27,52


In [194]:
df_waste["waste nn"] = [i if i > 0 else 0 for i in df_waste["waste"]]
waste_group = df_waste.groupby("week").sum()


In [195]:
waste_group["week"] = waste_group.index

In [196]:
import plotly.express as px
fig = px.line(waste_group, x = "week", y = "waste nn")
fig.show()

In [112]:
purchases_per_day

day
2018-01-01       8
2018-01-02      13
2018-01-03      18
2018-01-04      30
2018-01-05      40
              ... 
2018-12-26    2913
2018-12-27    2918
2018-12-28    2921
2018-12-29    2934
2018-12-30    2943
Name: product_id, Length: 364, dtype: int64

In [71]:
transactions["day"] = pd.to_datetime(transactions["day"])
filter_transactions = transactions[transactions["description"] == product]

In [86]:
daydate = df_date[df_date["day"] == 2]["date"].values[0]

filter_transactions[filter_transactions["day"] <= daydate]


,day,time,customer,bank acount,category,product_id,description,size,std_sales_price,purchase_price,bio,basic
89,2018-01-01,10:40:13,6.0,54211938.0,vegetable,v_1,Komkommer,1st,0.69,0.69,0.0,0.0
467,2018-01-01,14:21:35,26.0,51433968.0,vegetable,v_1,Komkommer,1st,0.69,0.69,0.0,0.0
573,2018-01-01,15:36:46,33.0,30794082.0,vegetable,v_1,Komkommer,1st,0.69,0.69,0.0,0.0
700,2018-01-01,16:56:20,39.0,15832817.0,vegetable,v_1,Komkommer,1st,0.69,0.69,0.0,0.0
808,2018-01-01,18:15:29,46.0,90399261.0,vegetable,v_1,Komkommer,1st,0.69,0.69,0.0,0.0
821,2018-01-01,18:15:29,46.0,90399261.0,vegetable,v_1,Komkommer,1st,0.69,0.69,0.0,0.0
824,2018-01-01,18:15:29,46.0,90399261.0,vegetable,v_1,Komkommer,1st,0.69,0.69,0.0,0.0
879,2018-01-01,18:46:48,49.0,15382238.0,vegetable,v_1,Komkommer,1st,0.69,0.69,0.0,0.0
29093,2018-01-02,12:52:06,1714.0,28719528.0,vegetable,v_1,Komkommer,1st,0.69,0.69,0.0,0.0
29134,2018-01-02,13:44:14,1718.0,39309224.0,vegetable,v_1,Komkommer,1st,0.69,0.69,0.0,0.0


In [111]:
purchases_per_day = filter_transactions.groupby("day").count()["product_id"].cumsum()
purchases_per_day

day
2018-01-01       8
2018-01-02      13
2018-01-03      18
2018-01-04      30
2018-01-05      40
              ... 
2018-12-26    2913
2018-12-27    2918
2018-12-28    2921
2018-12-29    2934
2018-12-30    2943
Name: product_id, Length: 364, dtype: int64

In [52]:
daydate

numpy.datetime64('2018-01-15T00:00:00.000000000')

In [74]:
dayofyear = list(range(365))
date = [datetime.datetime(2018, 1, 1) + datetime.timedelta(day) for day in dayofyear]
df_date = pd.DataFrame({"day":dayofyear, "date":date})
df_date

,day,date
0,0,2018-01-01
1,1,2018-01-02
2,2,2018-01-03
3,3,2018-01-04
4,4,2018-01-05
...,...,...
360,360,2018-12-27
361,361,2018-12-28
362,362,2018-12-29
363,363,2018-12-30
